<img src="vua.png">

# Klasteriranje

Strojno učenje možemo provoditi na dva osnovna načina:
- **nadzirano** učenje (engl. *supervised learning*)
- **nenadzirano** učenje (engl *unsupervised learning*)

Ako su nam poznati ulazni podaci i ciljne varijable tada se radi o nadziranom učenju prilikom kojeg koristimo **regresiju** za kontinuirane/brojčane vrijednosti ili **klasifikaciju** za diskretne/nebrojčane vrijednosti.

Ako želimo pronaći pravilnost u podacima i od modela očekujemo da sam otkrije vrijednosti ciljnih varijabli tada se radi o nenadziranom učenju prilikom kojeg između ostalog koristimo **klasteriranje** (grupiranje, engl. *clustering*) i **smanjenje dimenzionalnosti** (engl. *dimensionality reduction*).

||kontinuirane varijable|kategoričke varijable|
|---|---|---|
|**nadzirano učenje**|regresija|klasifikacija|
|**nenadzirano učenje**|smanjenje dimenzionalnosti|**klasteriranje**|


Tema ovih vježbi je **klasteriranje**:

U poslovnoj praksi se možemo naći u situaciji rješavanja nekog problema na temelju podataka, ali bez definiranih izlaza ili ciljne varijable. Često želimo grupirati podatke, ali ne znamo na temelju kojih značajki ili u koliko ih je grupa optimalno podijeliti. Nemamo prethodno znanje što u njima tražiti. U takvim slučajevima tražimo sličnosti na temelju kojih bismo podatke mogli grupirati odnosno kreirati **klastere**.

Tehnike klasteriranja služe formiranju grupa podataka sličnih vrijednosti. U ovim vježbama ćemo koristiti tehniku "k-Means". Prednost joj je da je jednostavna za razumijevanje, a nedostatak da je računalno dosta zahtjevna i sporo konvergira (dolazi do rješenja).


Za početak ćemo pripremiti neophodne biblioteke funkcija. Novi objekti koji dosad nisu korišteni u vježbama su **kmeans**, **vq** i **cdist** iz biblioteke **scipy**. Više detalja možete pronaći na adresama:
> https://docs.scipy.org/doc/scipy-0.18.1/reference/cluster.vq.html
> https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.cdist.html


Za razliku od nadziranog učenja, za koje smo koristili podatke s opservacijama koje pripadaju nekoj kategoriji s ciljem da napravimo model koji će učiti na primjerima, u ovom slučaju nemamo ciljnu varijablu već nam je zadatak pronaći znanje skriveno u podacima.

### Primjer

Razmotrimo jedan od poznatih skupova podataka **iris** i klastere zasnovane na osobinama cvjetova. 

Postoje tri vrste irisa: *setosa* (plavo), *virginica* (zeleno) i *versicolor* (crveno).

Pogledajmo izvorne podatke i dijagram rasipanja (engl. *scatterplot*) za varijable 'SepalLength' i 'PetalLength'.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv('iris.csv')
print(df.shape, '\n', df.describe())

setosa = df[df.Species=='Iris-setosa']
versicolor = df[df.Species=='Iris-versicolor']
virginica = df[df.Species=='Iris-virginica']

%pylab inline

se = plt.scatter(setosa['SepalLength'], setosa['PetalLength'], marker='o', color='b', label='setosa')
vi = plt.scatter(virginica['SepalLength'],virginica['PetalLength'], marker='o', color='g', label='virginica')
ve = plt.scatter(versicolor['SepalLength'],versicolor['PetalLength'], marker='o', color='r', label='versicolor')
leg = plt.legend(handles=[se, vi, ve], frameon=False, loc='lower right')

Možemo uočiti da se stvaraju klasteri sa sličnim vrijednostima opservacija. Ovu ilustraciju koristimo zato što znamo da postoje tri vrste irisa i prirodno je da su osobine pojedine vrste slične i da će završiti u zasebnim klasterima.

U slučaju u kojem unaprijed ne znamo broj klastera, potrebno je napraviti analizu klastera. Također, klasteriranje nije uvijek ovisno o atributima (osobinama) koje koristimo, pa je potrebno istražiti različite kombinacije.

### Vježba

Na osnovu gornjeg primjera s varijablama *SepalLength* i *PetalLength* grafički prikažite klastere za kombinaciju varijabli **SepalLength** i **SepalWidth**.

In [ ]:
#ovdje upišite kôd koji rješava zadatak



Postoji li razlika u odnosu na prethodni primjer?

Odgovor:

Možemo li unaprijed znati koje atribute treba izabrati?  

Odgovor:

Koristi li se domensko znanje (poznavanje karakteristika cvijeća) pri odabiru atributa za klasteriranje?

Odgovor:


### Primjena

K-Means klastering je koristan u početnom istraživanju podataka kad nemamo ciljnu varijablu, a neki od primjera su:

* **Prepoznavanje pojmova u tekstu**. Kako primjerice prepoznati da li riječ "jaguar" predstavlja automobil ili životinju? Ako su riječi oko nje primjerice u istom klasteru s riječima "leopard" i "plijen" u odnosu na riječi "kilometri" i "kotači" možemo zaključiti da se radi o životinji. Ovo nam omogućuje klasteriranje dokumenata.

* **Demografski podaci**.  Klasteriranjem podataka o ponašanju korisnika neke usluge možemo grupirati uzorke te ih iskoristiti pri kreiranju prediktivnih modela prilagođenih svakom od klastera.

* ...

### Podaci UN-a o državama svijeta

U sljedećem primjeru ćemo potražiti korisne atribute u podacima iz datoteke UN.csv.

> Napomena: Obratite pozornost na kvalitetu podataka (nedostajuće vrijednosti). Budući da imamo samo jednu opservaciju za svaku državu ne smijemo maknuti one s nedostajućim vrijednostima jer ćemo ostati bez većine podataka!

Pogledajmo kako izgledaju podaci:

In [ ]:
import pandas as pd
df = pd.read_csv('UN.csv')
print(df[:10])

Pogledajmo koliko podataka nam je na raspolaganju za koje varijable:

In [ ]:
print(df.count())

Vidimo 14 atributa (**country** i **region** su *string*, a ostale *float*). Imamo ukupno 207 opservacija za 207 država. Osim države i regije koje imaju po 207 opservacija ostali atributi imaju nedostajućih vrijednosti.
Kako ne želimo ostati bez većine država uzimamo samo atribute koji imaju malo nedostajućih vrijednosti (**tfr**, **lifeMale**, **lifeFemale**, **GDP**, **infantMortality**) i tako ostati bez samo par država.
Da smo primjerice koristili **educationMale** i **educationFemale** ostali bismo bez 2/3 podataka. 

Novi skup podataka imat će atribute: **country**, **region**, **tfr**, **lifeMale**, **lifeFemale**, **GDP** i **infantMortality**. Intuitivno možemo pretpostaviti da vrijednosti varijabli **lifeMale**, **lifeFemale** i **infantMortality** u odnosu na **GDP** mogu dati relevantan rezultat.  

Budući da unaprijed ne znamo na koliko klastera bi bilo optimalno podijeliti podatke, pokušajmo, za početak, napraviti klastere različitih kardinalnosti.

Koristit ćemo biblioteke **NumPy** i **SciPy** za *k-Means klastering* i to s k = 1 do 10 (broj klastera).  

Za vježbu možemo pretpostaviti da je broj klastera manji od 10 i kreirati dijagram za odabir broja klastera iz kojeg možemo vidjeti koliko su klasteri dobro grupirani. Izračunajmo sumu kvadratnih udaljenosti svakog člana klastera od centra klastera pa to ponovimo za svaki klaster. U "dobrim" klasterima će udaljenosti biti manje, ali treba voditi računa o tome da će se udaljenosti smanjivati i povećanjem broja klastera. Promjena udaljenosti nije linearna pa uzimamo za *k* (broj klastera) zadnju vrijednost kod koje je pad bio značajan.

Vratimo se podacima:  

In [ ]:
# Učitavamo dokument u kojem su samo 4 atributa lifeMale, lifeFemale, infantMortality i GDPperCapita

fName = ('UN4col.csv')
fp = open(fName)
X = np.loadtxt(fp)
fp.close()
X

In [ ]:
import numpy as np
from scipy.cluster.vq import kmeans,vq
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt

##### Klasteriramo podatke u  K=1..10 klastera #####
K = range(1,10)

KM = [kmeans(X,k) for k in K] # apply kmeans 1 to 10
centroids = [cent for (cent,var) in KM]   # cluster centroids

D_k = [cdist(X, cent, 'euclidean') for cent in centroids]

cIdx = [np.argmin(D,axis=1) for D in D_k]
dist = [np.min(D,axis=1) for D in D_k]
avgWithinSS = [sum(d)/X.shape[0] for d in dist]  

In [ ]:
kIdx = 2
# plot elbow curve
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(K, avgWithinSS, 'b*-')
ax.plot(K[kIdx], avgWithinSS[kIdx], marker='o', markersize=12, 
      markeredgewidth=2, markeredgecolor='r', markerfacecolor='None')
plt.grid(True)
plt.xlabel('Broj klastera')
plt.ylabel('Prosječna suma kvadrata udaljenosti')
tt = plt.title('Elbow - k-Means klasteriranje')  

Vidimo da bi 3 bio dobar broj klastera.
Sad možemo iskoristiti k-Means algoritam za treniranje klastera primjerice za **Infant Mortality** i **GDP**.

Prije nastavka, pogledajte kôd u datoteci **kmeans.py**, gdje se nalaze funkcije korištene kod sljedećih vizualizacija.

In [ ]:
from sklearn.cluster import KMeans
km = KMeans(3, init='k-means++')
km.fit(X)
c = km.predict(X)

In [ ]:
import kmeans as mykm
(pl0,pl1,pl2) = mykm.plot_clusters(X,c,3,2)

Iz grafičkog prikaza je vidljivo da države s manjim GDP-om imaju veću smrtnost beba, a rastom GDP-a smrtnost pada.

Vidimo tri klastera koja i po GDP-u možemo odmah nazvati nerazvijene zemlje, zemlje u razvoju i razvijene zemlje.

Probajmo neki drugi atribut - **lifeMale**...

In [ ]:
(pl0,pl1,pl2) = mykm.plot_clusters(X,c,3,0,False)

... i **lifeFemale**

In [ ]:
(pl0,pl1,pl2) = mykm.plot_clusters(X,c,3,1,False)

U zadnja dva prikaza vidimo da i dalje imamo klastere koji podjednako dijele uzorak po GDP-u, ali s drugim trendom (što je i očekivano).

### Zadatak
Prođite kroz kod i odgovorite na pitanja.  

In [ ]:
import numpy as np
from scipy.cluster.vq import kmeans
from scipy.spatial.distance import cdist,pdist
from sklearn import datasets
from sklearn.decomposition import PCA
from matplotlib import pyplot as plt
from matplotlib import cm

##### data #####
# load digits dataset
data = datasets.load_digits()
t = data['target']

In [ ]:
# Posjetite link
# https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_digits.html
# O kakvim se podacima radi, koliko ima uzoraka i koliko nezavisnih varijabli:


In [ ]:
# perform PCA dimensionality reduction
pca = PCA(n_components=2,svd_solver='randomized').fit(data['data'])
X = pca.transform(data['data'])

In [ ]:
# Posjetite link
# https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html
# Što se dogodilo s podacima i što je napravio PCA



In [ ]:
##### cluster data into K=1..20 clusters #####
K_MAX = 20
KK = range(1,K_MAX+1)

KM = [kmeans(X,k) for k in KK]
centroids = [cent for (cent,var) in KM]
D_k = [cdist(X, cent, 'euclidean') for cent in centroids]
cIdx = [np.argmin(D,axis=1) for D in D_k]
dist = [np.min(D,axis=1) for D in D_k]

tot_withinss = [sum(d**2) for d in dist]  # Total within-cluster sum of squares
totss = sum(pdist(X)**2)/X.shape[0]       # The total sum of squares
betweenss = totss - tot_withinss          # The between-cluster sum of squares


In [ ]:
# Posjetite link
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.vq.vq.html#scipy.cluster.vq.vq
# Što radi kmeans, što predstavljaju "centroids"?



In [ ]:
##### plots #####
kIdx = 9        # K=10
clr = cm.Spectral( np.linspace(0,1,10) ).tolist()
mrk = 'os^p<dvh8>+x.'

# elbow curve
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(KK, betweenss/totss*100, 'b*-')
ax.plot(KK[kIdx], betweenss[kIdx]/totss*100, marker='o', markersize=12, 
    markeredgewidth=2, markeredgecolor='r', markerfacecolor='None')
ax.set_ylim((0,100))
plt.grid(True)
plt.xlabel('Number of clusters')
plt.ylabel('Percentage of variance explained (%)')
plt.title('Elbow for KMeans clustering')
plt.show()

In [ ]:
# Posjetite link
# https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_silhouette_analysis.html
# Kako koristimo elbow krivulju?

In [ ]:
# show centroids for K=10 clusters
plt.figure()
for i in range(kIdx+1):
    img = pca.inverse_transform(centroids[kIdx][i]).reshape(8,8)
    ax = plt.subplot(3,4,i+1)
    ax.set_xticks([])
    ax.set_yticks([])
    plt.imshow(img, cmap=cm.gray)
    plt.title( 'Cluster %d' % i )
plt.show()

In [ ]:
# dodano da ne javlja upozorenja
from matplotlib.axes._axes import _log as matplotlib_axes_logger
matplotlib_axes_logger.setLevel('ERROR')
# --------------------------------------------------------------

# compare K=10 clustering vs. actual digits
fig = plt.figure(figsize=(16, 12))
ax = fig.add_subplot(121)
for i in range(10):
    ind = (t==i)
    ax.scatter(X[ind,0],X[ind,1], s=35, c=clr[i], marker=mrk[i], label='%d'%i)
plt.legend()
plt.title('Actual Digits')
ax = fig.add_subplot(122)
for i in range(kIdx+1):
    ind = (cIdx[kIdx]==i)
    ax.scatter(X[ind,0],X[ind,1], s=35, c=clr[i], marker=mrk[i], label='C%d'%i)
plt.legend()
plt.title('K=%d clusters'%KK[kIdx])

plt.show()

In [ ]:
# Da li klustering algoritam može dodjeliti vrijednost klasi i zašto?


